In [4]:
from datetime import datetime, timedelta
import pandas as pd
from io import StringIO
import requests
import pandahouse

In [5]:
connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator'
}

In [6]:
 q='''
        select
        toDate(time) as event_date, 
        user_id as user,
        gender,
        age,
        os,
        countIf(action, action='like') likes,
        countIf(action, action='view') views
        from simulator_20220320.feed_actions  
        where toDate(time)>=yesterday()
        group by event_date,user,gender,age,os'''
        
df_feed = pandahouse.read_clickhouse(q, connection=connection)

In [7]:
df_feed

,event_date,user,gender,age,os,likes,views
0,2022-04-21,39222,1,24,iOS,5,15
1,2022-04-20,5673,0,18,Android,2,26
2,2022-04-20,4820,1,23,Android,4,17
3,2022-04-20,66899,0,26,Android,5,28
4,2022-04-21,157215,1,44,iOS,9,44
...,...,...,...,...,...,...,...
37268,2022-04-21,107579,1,22,Android,4,13
37269,2022-04-20,151879,0,40,iOS,10,40
37270,2022-04-20,154085,1,24,iOS,2,12
37271,2022-04-20,67911,0,44,Android,5,35


In [8]:
q2='''
        with sent as (
        select 
        toDate(time) as event_date,
        user_id as user,
        gender,
        age,
        os,
        count(distinct reciever_id) as sent_users,
        count(reciever_id) as sent_messages
        from simulator_20220320.message_actions
        where event_date = yesterday() 
        group by event_date,user,gender,age,os
        ),
        received as (
        select 
        toDate(time) as event_date,
        reciever_id as user,

        count(distinct user_id) as received_users,
        count(user_id) as received_messages
        from simulator_20220320.message_actions
        where event_date = yesterday() 
        group by event_date,user

        )
        select 
        a.event_date,
        a.user,
        gender,
        age,
        os,
        sent_users,
        sent_messages,
        received_users,
        received_messages
        from sent a inner join received b on a.user=b.user '''
        
df_messages = pandahouse.read_clickhouse(q2, connection=connection)


In [9]:
df_messages

,event_date,user,gender,age,os,sent_users,sent_messages,received_users,received_messages
0,2022-04-20,18991,1,19,Android,4,4,4,4
1,2022-04-20,109233,1,33,iOS,2,2,4,4
2,2022-04-20,7998,1,40,Android,1,2,2,2
3,2022-04-20,15087,1,26,iOS,5,5,2,2
4,2022-04-20,17480,0,23,Android,9,10,14,16
...,...,...,...,...,...,...,...,...,...
1615,2022-04-20,111929,0,25,Android,11,12,18,20
1616,2022-04-20,18879,1,22,iOS,2,2,7,7
1617,2022-04-20,115990,0,29,Android,5,5,12,12
1618,2022-04-20,125552,1,25,Android,2,2,2,2


In [12]:
 df = pd.merge(df_feed, df_messages, on=['event_date','user','gender','age','os'],how = 'outer')

In [13]:
df

,event_date,user,gender,age,os,likes,views,sent_users,sent_messages,received_users,received_messages
0,2022-04-21,39222,1,24,iOS,5.0,15.0,NaN,NaN,NaN,NaN
1,2022-04-20,5673,0,18,Android,2.0,26.0,NaN,NaN,NaN,NaN
2,2022-04-20,4820,1,23,Android,4.0,17.0,NaN,NaN,NaN,NaN
3,2022-04-20,66899,0,26,Android,5.0,28.0,NaN,NaN,NaN,NaN
4,2022-04-21,157215,1,44,iOS,9.0,44.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
38676,2022-04-20,108509,1,22,Android,NaN,NaN,3.0,3.0,12.0,13.0
38677,2022-04-20,111929,0,25,Android,NaN,NaN,11.0,12.0,18.0,20.0
38678,2022-04-20,115990,0,29,Android,NaN,NaN,5.0,5.0,12.0,12.0
38679,2022-04-20,125552,1,25,Android,NaN,NaN,2.0,2.0,2.0,2.0


In [45]:
 df_final = df[['event_date', 'gender', 'age', 'os', 'views', 'likes', 
                          'received_messages', 'sent_messages', 'received_users', 'sent_users']]\
            .groupby(['event_date', 'gender', 'age', 'os'], as_index=False)\
            .sum()
    

In [40]:
df_final

,event_date,gender,age,os,views,likes,received_messages,sent_messages,received_users,sent_users
0,2022-04-20,0,14,Android,2762.0,606.0,50.0,56.0,44.0,28.0
1,2022-04-20,0,14,iOS,1504.0,336.0,16.0,28.0,15.0,14.0
2,2022-04-20,0,15,Android,5552.0,1167.0,33.0,65.0,31.0,45.0
3,2022-04-20,0,15,iOS,4046.0,865.0,29.0,41.0,28.0,34.0
4,2022-04-20,0,16,Android,7488.0,1647.0,138.0,120.0,126.0,111.0
...,...,...,...,...,...,...,...,...,...,...
457,2022-04-21,1,70,iOS,15.0,1.0,0.0,0.0,0.0,0.0
458,2022-04-21,1,71,Android,31.0,7.0,0.0,0.0,0.0,0.0
459,2022-04-21,1,72,iOS,11.0,1.0,0.0,0.0,0.0,0.0
460,2022-04-21,1,74,Android,12.0,3.0,0.0,0.0,0.0,0.0


In [42]:
    df_final[['likes', 'views', 'sent_messages', 'sent_users', 'received_messages', 'received_users']] = \
            df_final[['likes', 'views', 'sent_messages', 'sent_users', 'received_messages', 'received_users']].astype(int)

In [43]:
df_final

,event_date,gender,age,os,views,likes,received_messages,sent_messages,received_users,sent_users
0,2022-04-20,0,14,Android,2762,606,50,56,44,28
1,2022-04-20,0,14,iOS,1504,336,16,28,15,14
2,2022-04-20,0,15,Android,5552,1167,33,65,31,45
3,2022-04-20,0,15,iOS,4046,865,29,41,28,34
4,2022-04-20,0,16,Android,7488,1647,138,120,126,111
...,...,...,...,...,...,...,...,...,...,...
457,2022-04-21,1,70,iOS,15,1,0,0,0,0
458,2022-04-21,1,71,Android,31,7,0,0,0,0
459,2022-04-21,1,72,iOS,11,1,0,0,0,0
460,2022-04-21,1,74,Android,12,3,0,0,0,0


In [17]:
os = df.groupby('os').agg({'event_date':'min', \
                                    'likes':'sum', \
                                    'views': 'sum', \
                                    'received_messages':'sum', \
                                    'received_users':'sum', \
                                    'sent_messages':'sum', \
                                    'sent_users':'sum'}).reset_index().copy()

In [18]:
os

,os,event_date,likes,views,received_messages,received_users,sent_messages,sent_users
0,Android,2022-04-20,162989.0,740385.0,8454.0,6692.0,5346.0,4600.0
1,iOS,2022-04-20,89320.0,404366.0,4268.0,3445.0,2801.0,2390.0


In [19]:
os['metric'] = 'os'


In [20]:
os

,os,event_date,likes,views,received_messages,received_users,sent_messages,sent_users,metric
0,Android,2022-04-20,162989.0,740385.0,8454.0,6692.0,5346.0,4600.0,os
1,iOS,2022-04-20,89320.0,404366.0,4268.0,3445.0,2801.0,2390.0,os


In [21]:
os.rename(columns={'os':'metric_value'},inplace=True)

In [22]:
os

,metric_value,event_date,likes,views,received_messages,received_users,sent_messages,sent_users,metric
0,Android,2022-04-20,162989.0,740385.0,8454.0,6692.0,5346.0,4600.0,os
1,iOS,2022-04-20,89320.0,404366.0,4268.0,3445.0,2801.0,2390.0,os


In [25]:
gender = df.groupby('gender').agg({'event_date':'min', \
                                    'likes':'sum', \
                                    'views': 'sum', \
                                    'received_messages':'sum', \
                                    'received_users':'sum', \
                                    'sent_messages':'sum', \
                                    'sent_users':'sum'}).reset_index().copy()
gender['metric'] = 'gender'
gender.rename(columns={'gender':'metric_value'},inplace=True)

In [26]:
gender

,metric_value,event_date,likes,views,received_messages,received_users,sent_messages,sent_users,metric
0,0,2022-04-20,114105.0,515795.0,6200.0,4825.0,3845.0,3253.0,gender
1,1,2022-04-20,138204.0,628956.0,6522.0,5312.0,4302.0,3737.0,gender


In [27]:
age = df.groupby('age').agg({'event_date':'min', \
                                    'likes':'sum', \
                                    'views': 'sum', \
                                     'received_messages':'sum', \
                                    'received_users':'sum', \
                                    'sent_messages':'sum', \
                                    'sent_users':'sum'}).reset_index().copy()
age['metric'] = 'age'
age.rename(columns={'age':'metric_value'},inplace=True)

In [28]:
age

,metric_value,event_date,likes,views,received_messages,received_users,sent_messages,sent_users,metric
0,14,2022-04-20,3358.0,15372.0,116.0,108.0,131.0,64.0,age
1,15,2022-04-20,8208.0,38294.0,187.0,172.0,288.0,221.0,age
2,16,2022-04-20,9319.0,42199.0,377.0,336.0,306.0,277.0,age
3,17,2022-04-20,12050.0,54719.0,568.0,474.0,426.0,400.0,age
4,18,2022-04-20,14408.0,65909.0,824.0,640.0,500.0,483.0,age
...,...,...,...,...,...,...,...,...,...
61,75,2022-04-20,4.0,12.0,0.0,0.0,0.0,0.0,age
62,76,2022-04-20,3.0,17.0,0.0,0.0,0.0,0.0,age
63,77,2022-04-20,17.0,73.0,0.0,0.0,0.0,0.0,age
64,79,2022-04-20,13.0,51.0,0.0,0.0,0.0,0.0,age


In [29]:
 union_metrics = pd.concat([os, gender, age])

In [30]:
union_metrics

,metric_value,event_date,likes,views,received_messages,received_users,sent_messages,sent_users,metric
0,Android,2022-04-20,162989.0,740385.0,8454.0,6692.0,5346.0,4600.0,os
1,iOS,2022-04-20,89320.0,404366.0,4268.0,3445.0,2801.0,2390.0,os
0,0,2022-04-20,114105.0,515795.0,6200.0,4825.0,3845.0,3253.0,gender
1,1,2022-04-20,138204.0,628956.0,6522.0,5312.0,4302.0,3737.0,gender
0,14,2022-04-20,3358.0,15372.0,116.0,108.0,131.0,64.0,age
...,...,...,...,...,...,...,...,...,...
61,75,2022-04-20,4.0,12.0,0.0,0.0,0.0,0.0,age
62,76,2022-04-20,3.0,17.0,0.0,0.0,0.0,0.0,age
63,77,2022-04-20,17.0,73.0,0.0,0.0,0.0,0.0,age
64,79,2022-04-20,13.0,51.0,0.0,0.0,0.0,0.0,age


In [31]:
new_cols = ['event_date',
                    'metric',
                    'metric_value',
                    'views',
                    'likes',
                    'received_messages',
                    'sent_messages',
                    'received_users',
                    'sent_users']  

In [32]:
new_cols

['event_date',
 'metric',
 'metric_value',
 'views',
 'likes',
 'received_messages',
 'sent_messages',
 'received_users',
 'sent_users']

In [33]:
final_metrics = union_metrics.loc[:, new_cols]
final_metrics

,event_date,metric,metric_value,views,likes,received_messages,sent_messages,received_users,sent_users
0,2022-04-20,os,Android,740385.0,162989.0,8454.0,5346.0,6692.0,4600.0
1,2022-04-20,os,iOS,404366.0,89320.0,4268.0,2801.0,3445.0,2390.0
0,2022-04-20,gender,0,515795.0,114105.0,6200.0,3845.0,4825.0,3253.0
1,2022-04-20,gender,1,628956.0,138204.0,6522.0,4302.0,5312.0,3737.0
0,2022-04-20,age,14,15372.0,3358.0,116.0,131.0,108.0,64.0
...,...,...,...,...,...,...,...,...,...
61,2022-04-20,age,75,12.0,4.0,0.0,0.0,0.0,0.0
62,2022-04-20,age,76,17.0,3.0,0.0,0.0,0.0,0.0
63,2022-04-20,age,77,73.0,17.0,0.0,0.0,0.0,0.0
64,2022-04-20,age,79,51.0,13.0,0.0,0.0,0.0,0.0


In [34]:
final_metrics = final_metrics.reset_index().drop('index', axis =1)
final_metrics

,event_date,metric,metric_value,views,likes,received_messages,sent_messages,received_users,sent_users
0,2022-04-20,os,Android,740385.0,162989.0,8454.0,5346.0,6692.0,4600.0
1,2022-04-20,os,iOS,404366.0,89320.0,4268.0,2801.0,3445.0,2390.0
2,2022-04-20,gender,0,515795.0,114105.0,6200.0,3845.0,4825.0,3253.0
3,2022-04-20,gender,1,628956.0,138204.0,6522.0,4302.0,5312.0,3737.0
4,2022-04-20,age,14,15372.0,3358.0,116.0,131.0,108.0,64.0
...,...,...,...,...,...,...,...,...,...
65,2022-04-20,age,75,12.0,4.0,0.0,0.0,0.0,0.0
66,2022-04-20,age,76,17.0,3.0,0.0,0.0,0.0,0.0
67,2022-04-20,age,77,73.0,17.0,0.0,0.0,0.0,0.0
68,2022-04-20,age,79,51.0,13.0,0.0,0.0,0.0,0.0


In [47]:
final_metrics['event_date']=final_metrics['event_date'].apply(lambda x: datetime.isoformat(x))
final_metrics

TypeError: descriptor 'isoformat' for 'datetime.datetime' objects doesn't apply to a 'str' object

In [48]:
final_metrics = final_metrics.astype({
                       'views':'int', \
                       'likes':'int', \
                       'messages_recieved':'int', \
                       'messages_sent':'int', \
                       'users_received':'int', \
                       'users_sent':'int'})    

KeyError: 'Only a column name can be used for the key in a dtype mappings argument.'